In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 3
LAYER_SIZE = 10
S_LAYER_SIZE = 15
LATENT_SIZE = 4

In [3]:
# Load the data from the file

saved = np.load("testing1.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.Size([50000, 11])


11

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.l2a = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
#     x = 
#     y = None
    
    
    
    
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
train_data=train_data[torch.randperm(train_data.size()[0])]
test_data=test_data[torch.randperm(test_data.size()[0])]

In [10]:
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=3e-4)

In [11]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

Epoch: 0, Loss: 7.509440319824218
Epoch: 1, Loss: 7.183370754394531
Epoch: 2, Loss: 7.0088898828125


In [12]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [13]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [14]:
from torch.autograd import Variable

print("Action")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 1).to("cuda")
    print(sample[0][5])
    x = vae1(sample)
    print(x[0][0][5], '\n')

Action
tensor(0.6896, device='cuda:0')
tensor(0.4890, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1875, device='cuda:0')
tensor(0.5019, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.6406, device='cuda:0')
tensor(0.4993, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.7215, device='cuda:0')
tensor(0.5094, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.6110, device='cuda:0')
tensor(0.5057, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.3201, device='cuda:0')
tensor(0.4967, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0472, device='cuda:0')
tensor(0.5109, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.7684, device='cuda:0')
tensor(0.4990, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4446, device='cuda:0')
tensor(0.5087, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8828, device='cuda:0')
tensor(0.4969, device='cuda:0', grad_fn=<SelectBackward>) 



In [15]:
print("Reward")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 0.15).to("cuda")
    print(sample[0][9])
    x = vae1(sample)
    print(x[0][0][9], '\n')

Reward
tensor(0.0844, device='cuda:0')
tensor(0.0875, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1331, device='cuda:0')
tensor(0.1024, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0859, device='cuda:0')
tensor(0.1442, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0629, device='cuda:0')
tensor(0.2584, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1313, device='cuda:0')
tensor(0.1967, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1076, device='cuda:0')
tensor(0.0734, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1192, device='cuda:0')
tensor(0.1632, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0221, device='cuda:0')
tensor(0.1424, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0138, device='cuda:0')
tensor(0.1750, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0169, device='cuda:0')
tensor(0.1519, device='cuda:0', grad_fn=<SelectBackward>) 



In [19]:
torch.set_printoptions(linewidth=140)
for i in range(5):
    sample = torch.FloatTensor(1, 11).uniform_(0, 1).to("cuda")
    x, m, s = vae1(sample)
    loss = loss_function(x, sample, m, s)
    
    x = x.to("cpu")
    sample = sample.to("cpu")
    
    print(f"Original:       {sample}")
    print(f"Reconstruction: {x}")
    print(f"Loss: {loss}\n\n")
    

Original:       tensor([[0.3326, 0.0023, 0.2534, 0.9361, 0.7955, 0.7620, 0.2810, 0.3772, 0.0728, 0.2364, 0.4963]])
Reconstruction: tensor([[0.4516, 0.5441, 0.4798, 0.5115, 0.5390, 0.4983, 0.5348, 0.4998, 0.4904, 0.1753, 0.3520]], grad_fn=<CopyBackwards>)
Loss: 7.537787437438965


Original:       tensor([[0.3061, 0.6161, 0.4320, 0.6255, 0.6189, 0.5128, 0.3723, 0.6714, 0.1396, 0.2378, 0.1113]])
Reconstruction: tensor([[0.4987, 0.5101, 0.5079, 0.5022, 0.4979, 0.4932, 0.4817, 0.5064, 0.5077, 0.1217, 0.2193]], grad_fn=<CopyBackwards>)
Loss: 7.22882080078125


Original:       tensor([[0.6172, 0.6287, 0.7405, 0.3829, 0.9664, 0.0034, 0.9259, 0.0628, 0.0771, 0.8421, 0.8242]])
Reconstruction: tensor([[0.5107, 0.5062, 0.4897, 0.4747, 0.5014, 0.5015, 0.5037, 0.4838, 0.4935, 0.1738, 0.3061]], grad_fn=<CopyBackwards>)
Loss: 8.753585815429688


Original:       tensor([[0.2213, 0.9644, 0.4720, 0.8401, 0.2147, 0.2597, 0.8559, 0.9579, 0.4764, 0.5699, 0.0090]])
Reconstruction: tensor([[0.4931, 0.5175, 0.

In [17]:
for i in range(99):
    print(test_data[i])

tensor([0.5002, 0.5004, 0.5002, 0.5002, 0.7774, 0.5009, 0.4989, 0.5306, 0.4299,
        0.0500, 0.0000])
tensor([0.5003, 0.4996, 0.5001, 0.5002, 0.3954, 0.5001, 0.5001, 0.4887, 0.5265,
        0.0500, 0.0000])
tensor([0.5001, 0.5001, 0.5001, 0.5004, 0.3878, 0.5052, 0.4886, 0.5533, 0.3803,
        0.0500, 1.0000])
tensor([0.5005, 0.4999, 0.5004, 0.5000, 0.0215, 0.4994, 0.5024, 0.4479, 0.6221,
        0.0500, 0.0000])
tensor([0.4995, 0.4996, 0.4997, 0.5004, 0.7333, 0.4996, 0.4993, 0.5187, 0.4552,
        0.0500, 0.0000])
tensor([0.5002, 0.4997, 0.4996, 0.4995, 0.4629, 0.5041, 0.4906, 0.5511, 0.3823,
        0.0500, 0.0000])
tensor([0.5001, 0.4996, 0.5003, 0.5002, 0.7037, 0.5024, 0.4944, 0.5590, 0.3668,
        0.0500, 0.0000])
tensor([0.5002, 0.5004, 0.4999, 0.4997, 0.6960, 0.4978, 0.5059, 0.4744, 0.5571,
        0.0500, 0.0000])
tensor([0.4998, 0.4999, 0.4999, 0.5004, 0.9147, 0.4978, 0.5043, 0.5153, 0.4636,
        0.0500, 0.0000])
tensor([0.5004, 0.5003, 0.5003, 0.5000, 0.5127, 0.5045,

In [18]:
vae1(test_data[0].to(device))

(tensor([0.5037, 0.5006, 0.4984, 0.4855, 0.5062, 0.5058, 0.5164, 0.4833, 0.4912,
         0.1900, 0.3372], device='cuda:0', grad_fn=<SigmoidBackward>),
 tensor([ 0.0075, -0.0646,  0.0115,  0.0794], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([0.0351, 0.0227, 0.0375, 0.0389], device='cuda:0',
        grad_fn=<AddBackward0>))